<h1>Chapter 1 - Introduction to Language Models</h1>
<i>Exploring the exciting field of Language AI</i>


<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961"><img src="https://img.shields.io/badge/Buy%20the%20Book!-grey?logo=amazon"></a>
<a href="https://www.oreilly.com/library/view/hands-on-large-language/9781098150952/"><img src="https://img.shields.io/badge/O'Reilly-white.svg?logo=data:image/svg%2bxml;base64,PHN2ZyB3aWR0aD0iMzQiIGhlaWdodD0iMjciIHZpZXdCb3g9IjAgMCAzNCAyNyIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPGNpcmNsZSBjeD0iMTMiIGN5PSIxNCIgcj0iMTEiIHN0cm9rZT0iI0Q0MDEwMSIgc3Ryb2tlLXdpZHRoPSI0Ii8+CjxjaXJjbGUgY3g9IjMwLjUiIGN5PSIzLjUiIHI9IjMuNSIgZmlsbD0iI0Q0MDEwMSIvPgo8L3N2Zz4K"></a>
<a href="https://github.com/HandsOnLLM/Hands-On-Large-Language-Models"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HandsOnLLM/Hands-On-Large-Language-Models/blob/main/chapter01/Chapter%201%20-%20Introduction%20to%20Language%20Models.ipynb)

---

This notebook is for Chapter 1 of the [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961) book by [Jay Alammar](https://www.linkedin.com/in/jalammar) and [Maarten Grootendorst](https://www.linkedin.com/in/mgrootendorst/).

---

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961">
<img src="https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/images/book_cover.png" width="350"/></a>


### [OPTIONAL] - Installing Packages on <img src="https://colab.google/static/images/icons/colab.png" width=100>

If you are viewing this notebook on Google Colab (or any other cloud vendor), you need to **uncomment and run** the following codeblock to install the dependencies for this chapter:

---

💡 **NOTE**: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---

In [2]:
# %%capture
# !pip install transformers>=4.40.1 accelerate>=0.27.2

# Phi-3

The first step is to load our model onto the GPU for faster inference. Note that we load the model and tokenizer separately (although that isn't always necessary).第一步是将我们的模型加载到 GPU 上以实现更快的推理。请注意，我们分别加载模型和分词器（尽管这并非总是必要的）。

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# AutoModelForCausalLM 是一个自动加载因果语言模型的类。因果语言模型是一种能够根据前面的文本预测下一个词的模型，常用于文本生成任务。
# AutoTokenizer 是一个自动加载分词器的类。分词器的作用是将输入的文本分割成一个个的词元（tokens），以便模型能够处理。
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", # 这里指定要加载的是微软发布的 Phi-3-mini-4k-instruct 模型。
    device_map="cuda", # 表示将模型加载到支持 CUDA 的 GPU 上进行计算
    torch_dtype="auto", # 表示自动选择合适的数据类型来存储模型的参数。不同的数据类型（如 float32、float16 等）在计算速度和内存占用上有所不同，自动选择可以根据硬件和任务的需求进行优化。
    trust_remote_code=True, # 表示信任远程代码。有些预训练模型可能包含一些自定义的代码，设置该参数为 True 可以允许加载和执行这些代码。
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct") # 确保分词器与模型是兼容的。

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Although we can now use the model and tokenizer directly, it's much easier to wrap it in a `pipeline` object:

In [9]:
from transformers import pipeline
# pipeline 是 transformers 库中一个非常方便的工具类，它为各种自然语言处理任务提供了一个高级的抽象接口。
# Create a pipeline
generator = pipeline(
    "text-generation", # 指定了管道的任务类型为文本生成。也就是说，这个管道将用于根据输入的文本生成后续的文本内容。
    model=model, # 将之前加载的预训练语言模型 model 传递给管道。这个模型将负责根据输入的文本生成新的文本。
    tokenizer=tokenizer,
    return_full_text=False, # 该参数控制管道返回的文本内容。当设置为 False 时，管道只返回模型生成的新文本，而不包含输入的原始文本；当设置为 True 时，返回的文本将包含原始输入和模型生成的内容。
    max_new_tokens=500,
    # 该参数控制文本生成的策略。
    # do_sample=False # 当设置为 False 时，模型将使用贪心搜索策略，即每次都选择概率最大的词作为下一个生成的词；
    do_sample=True # 当设置为 True 时，模型将使用采样策略，会根据词的概率分布随机选择下一个词，这样可以增加生成文本的多样性。
)

Device set to use cuda


Finally, we create our prompt as a user and give it to the model:

In [15]:
# The prompt (user input / query)
# messages 是一个列表，其中每个元素是一个字典。字典有两个键：
# role：表示消息的发送者角色，这里是 "user"，表明这是用户发送的消息。
# content：表示消息的具体内容，这里用户要求 “Create a funny joke about chickens.”，即生成一个关于鸡的有趣笑话。
messages = [
    {"role": "user", "content": "Create a funny joke about chickens."}
]

# Generate output
# generator 是之前使用 pipeline 创建的文本生成管道对象。这里将 messages 列表作为输入传递给 generator 进行文本生成。
output = generator(messages)
print(output[0]["generated_text"])

 Why do chickens make terrible singers? Because everyone knows they can't do anything without clucking!
